In [12]:
# Import Jobs to Firebase mit optimiertem "INSERT IGNORE"

import json
import os
import base64
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from hashlib import md5
from datetime import datetime

# 1. Lade die JSON-Datei mit allen Jobs
def load_jobs_from_json(file_path='jobs_all_processed.json'):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            jobs = json.load(f)
        print(f"Erfolgreich {len(jobs)} Jobs aus {file_path} geladen.")
        return jobs
    except Exception as e:
        print(f"Fehler beim Laden der JSON-Datei: {str(e)}")
        return []

# 2. Initialisiere Firebase mit Umgebungsvariable oder Datei
def setup_firebase():
    if firebase_admin._apps:
        return firestore.client()
        
    # Versuche die Umgebungsvariable zu nutzen
    if 'FIREBASE_SERVICE_ACCOUNT' in os.environ:
        try:
            service_account_base64 = os.environ['FIREBASE_SERVICE_ACCOUNT']
            service_account_json = base64.b64decode(service_account_base64)
            service_account_info = json.loads(service_account_json)
            cred = credentials.Certificate(service_account_info)
            firebase_admin.initialize_app(cred)
            print("Firebase mit Umgebungsvariable initialisiert.")
            return firestore.client()
        except Exception as e:
            print(f"Fehler bei der Verwendung der Umgebungsvariable: {str(e)}")
    
    # Ansonsten versuche die Datei zu nutzen
    cred_path = 'service-account.json'
    if os.path.exists(cred_path):
        cred = credentials.Certificate(cred_path)
        firebase_admin.initialize_app(cred)
        print("Firebase mit service-account.json initialisiert.")
        return firestore.client()
    else:
        print("Keine Firebase-Credentials gefunden.")
        return None

# 3. Ermittle vorhandene Job-IDs aus Firestore 
def get_existing_job_ids(db):
    """Holt die IDs aller bestehenden Jobs aus Firestore"""
    try:
        # Wir rufen nur die IDs ab, ohne die vollständigen Dokumente zu laden
        # Das spart erheblich Bandbreite
        print("Lade bestehende Job-IDs aus Firestore...")
        
        existing_ids = set()
        jobs_ref = db.collection('jobs')
        
        # Snapshots in Chunks abrufen (Firestore hat Limits für große Abfragen)
        # Wir verwenden nur die IDs, nicht die vollständigen Dokumente
        docs = jobs_ref.stream()
        for doc in docs:
            existing_ids.add(doc.id)
        
        print(f"Es wurden {len(existing_ids)} bestehende Job-IDs gefunden.")
        return existing_ids
    except Exception as e:
        print(f"Fehler beim Abrufen existierender Job-IDs: {str(e)}")
        return set()

# 4. Importiere Jobs in Firestore mit Batch-Processing und Skip von vorhandenen Jobs
def import_jobs_to_firestore(jobs, batch_size=400):
    db = setup_firebase()
    if not db:
        print("Firebase konnte nicht initialisiert werden.")
        return 0
    
    # Vorhandene Job-IDs abrufen
    existing_ids = get_existing_job_ids(db)
    
    # Anhand der IDs Jobs filtern, die importiert werden müssen
    jobs_to_import = []
    skipped_jobs = 0
    
    print("Identifiziere neue Jobs für den Import...")
    for job in jobs:
        if 'Link' in job:
            job_id = md5(job['Link'].encode()).hexdigest()
            if job_id not in existing_ids:
                jobs_to_import.append(job)
            else:
                skipped_jobs += 1
    
    print(f"{len(jobs_to_import)} neue Jobs zum Importieren gefunden.")
    print(f"{skipped_jobs} Jobs werden übersprungen, da sie bereits in Firestore vorhanden sind.")
    
    if not jobs_to_import:
        print("Keine neuen Jobs zu importieren. Import wird übersprungen.")
        return 0
    
    print(f"Beginne Import von {len(jobs_to_import)} neuen Jobs in Firestore...")
    total_imported = 0
    batches = 0
    
    # Führe den Import in Batches durch
    for i in range(0, len(jobs_to_import), batch_size):
        batch = db.batch()
        current_batch = jobs_to_import[i:i+batch_size]
        added_to_batch = 0
        
        print(f"Verarbeite Batch {batches+1} mit {len(current_batch)} Jobs...")
        
        for index, job in enumerate(current_batch):
            try:
                # Zeige den Fortschritt alle 50 Jobs an
                if index > 0 and index % 50 == 0:
                    print(f"  Fortschritt: {index}/{len(current_batch)} Jobs")
                
                # Erstelle eine eindeutige ID für den Job basierend auf dem Link
                job_id = md5(job['Link'].encode()).hexdigest()
                
                # Stelle sicher, dass adding_date existiert
                if 'adding_date' not in job:
                    job['adding_date'] = datetime.now().isoformat()
                
                # Füge die Dokument-ID zum Job hinzu
                job['firestore_id'] = job_id
                
                # Füge zum Batch hinzu
                doc_ref = db.collection('jobs').document(job_id)
                batch.set(doc_ref, job)
                added_to_batch += 1
                
            except Exception as e:
                print(f"Fehler bei Job {job.get('Title', 'Unbekannt')}: {str(e)}")
        
        # Commit den Batch
        if added_to_batch > 0:
            try:
                batch.commit()
                batches += 1
                total_imported += added_to_batch
                print(f"Batch {batches} abgeschlossen: {added_to_batch} Jobs importiert")
            except Exception as e:
                print(f"Fehler beim Commit von Batch {batches+1}: {str(e)}")
    
    print(f"Import abgeschlossen. Insgesamt {total_imported} neue Jobs in {batches} Batches importiert.")
    print(f"{skipped_jobs} vorhandene Jobs wurden übersprungen.")
    return total_imported

# 5. Überprüfe den Erfolg mit einer Abfrage
def verify_import(limit=5):
    db = setup_firebase()
    if not db:
        return
    
    print("Prüfe Ergebnis des Imports...")
    
    # Anzahl der Jobs in Firestore ermitteln
    jobs_ref = db.collection('jobs')
    
    # Anmerkung: In einer großen Datenbank sollte man nicht alle Dokumente abrufen,
    # aber für eine Schätzung verwenden wir eine effizientere Methode
    print("Zähle Anzahl der Dokumente in Firestore...")
    count_query = jobs_ref.count()
    count_result = count_query.get()
    jobs_count = count_result[0][0].value
    
    print(f"Anzahl Jobs in Firestore: {jobs_count}")
    print("\nStichprobe (bis zu 5 Jobs):")
    
    sample_jobs = list(jobs_ref.limit(limit).stream())
    for doc in sample_jobs:
        job_data = doc.to_dict()
        print(f"- {job_data.get('Title', 'Kein Titel')}")
        print(f"  ID: {doc.id}")
        print(f"  Link: {job_data.get('Link', 'Kein Link')}")
        print(f"  Hinzugefügt: {job_data.get('adding_date', 'Kein Datum')}")
        print("")

# Ausführen des Imports
# Lade die Jobs aus der JSON-Datei
jobs = load_jobs_from_json()

# Wenn Jobs vorhanden sind, in Firestore importieren
if jobs:
    imported_count = import_jobs_to_firestore(jobs)
    
    # Überprüfe den Erfolg
    if imported_count > 0:
        verify_import()
    elif imported_count == 0:
        print("Keine neuen Jobs importiert. Alle Jobs sind bereits vorhanden.")

Erfolgreich 4434 Jobs aus jobs_all_processed.json geladen.
Lade bestehende Job-IDs aus Firestore...
Es wurden 4429 bestehende Job-IDs gefunden.
Identifiziere neue Jobs für den Import...
5 neue Jobs zum Importieren gefunden.
4429 Jobs werden übersprungen, da sie bereits in Firestore vorhanden sind.
Beginne Import von 5 neuen Jobs in Firestore...
Verarbeite Batch 1 mit 5 Jobs...
Batch 1 abgeschlossen: 5 Jobs importiert
Import abgeschlossen. Insgesamt 5 neue Jobs in 1 Batches importiert.
4429 vorhandene Jobs wurden übersprungen.
Prüfe Ergebnis des Imports...
Zähle Anzahl der Dokumente in Firestore...
Anzahl Jobs in Firestore: 4434

Stichprobe (bis zu 5 Jobs):
- Responsable de programme (m/f) (réf. E00033260) (réf. F00033259)
  ID: 001f06d09d22ea49af5da6d4d4e09a2e
  Link: https://govjobs.public.lu/fr/postuler/postes-ouverts/postes-vacants/fonctionnaires/2025/A1/Janvier/20250106-responsabledeprogrammemfrfe000-336502.html
  Hinzugefügt: 2025-01-07T01:42:15.796717

- Expert en organisation (m